# Wrapper Routines

Since v1.0.10, following wrapper routines are introduced for easy access to other power flow and optimal power flow solvers. The wrapper routines are:
- PYPOWER: ``DCPF1``, ``PFlow1``, ``DCOPF1``, and ``ACOPF1``
- gurobi_optimods: ``OPF``

Following packages are required to call them:
- ``pypower``
- ``gurobi_optimods``

In [1]:
import numpy as np

import ams

In [2]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.json'),
              no_output=True)

## PYPOWER

### Power Flow

Detailed doc of ``PFlow1.run()`` is excerpted below.

In [3]:
help(sp.PFlow1.run)

Help on method run in module ams.routines.pypower:

run(**kwargs) method of ams.routines.pypower.PFlow1 instance
    Run the power flow using PYPOWER.

    Returns
    -------
    bool
        True if the optimization converged successfully, False otherwise.



In [4]:
sp.PFlow1.run()

Building system matrices
Parsing OModel for <PFlow1>
Evaluating OModel for <PFlow1>
Finalizing OModel for <PFlow1>
<PFlow1> converged in 0.0074 seconds, -1 iteration with PYPOWER.


PYPOWER Version 5.1.18, 10-Apr-2025 -- AC Power Flow (Newton)


Newton's method power flow converged in 3 iterations.


True

You can switch to other power flow algorithms.

Note that it is recommended to use ``update`` to update the config values.

In [5]:
sp.PFlow1.config.update(pf_alg=4)

In [6]:
sp.PFlow1.run()

<PFlow1> converged in 0.0189 seconds, -1 iteration with PYPOWER.


PYPOWER Version 5.1.18, 10-Apr-2025 -- AC Power Flow (Gauss-Seidel)


Gauss-Seidel power flow converged in 69 iterations.


True

After successful solving, the results are mapped back to routine variables.

In [7]:
sp.PFlow1.pg.v

array([1.02944468, 1.        , 3.2349    , 4.6651    , 0.1       ])

In [8]:
sp.PFlow1.qg.v

array([ 1.67217961, -0.02705768,  0.95026549, -0.40953028,  1.31140954])

Below is the active power on line flow

In [9]:
sp.PFlow1.plf.v

array([ 1.34144848,  1.16636881, -2.84926578, -0.22728223,  0.00756141,
       -1.80078873,  1.34144848])

A ``tcost`` object (an instance of ``ExpressionCalc``) is created in the power flow routines for quick evaluation of the total generation cost.

In [10]:
sp.PFlow1.tcost.v

3.0854544680774185

### Optimal Power Flow

In [11]:
sp.DCOPF1.run()

Parsing OModel for <DCOPF1>
Evaluating OModel for <DCOPF1>
Finalizing OModel for <DCOPF1>
<DCOPF1> converged in 0.0530 seconds, 10 iterations with PYPOWER.


PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


True

In PYPOWER, the ``c0`` term (the constant coefficient in the generator cost
function) is always included in the objective, regardless of the generator's
commitment status.

This means ``tcosts`` and ``obj.v`` can be different when some generators are
not committed.

See `pypower/opf_costfcn.py` for implementation details:
<https://rwl.github.io/PYPOWER/api/pypower.opf_costfcn-pysrc.html>

In [12]:
sp.DCOPF1.obj.e_str

'sum(c2 * pg**2 + c1 * pg + c0)'

In [13]:
sp.DCOPF1.obj.v

0.9585953752140786

In [14]:
sp.DCOPF1.tcost.e_str

'sum(mul(c2, pg**2))+ sum(mul(c1, pg))+ sum(mul(ug, c0))'

In [15]:
sp.DCOPF1.tcost.v

0.9585953752140786

### OPF

Similarly, both DCOPF1 and ACOPF1 are wrapped to solve optimal power flow.

In [16]:
help(sp.DCOPF1.run)

Help on method run in module ams.routines.pypower:

run(**kwargs) method of ams.routines.pypower.DCOPF1 instance
    Run the DCOPF routine using PYPOWER.

    Returns
    -------
    bool
        True if the optimization converged successfully, False otherwise.



In [17]:
sp.DCOPF1.run()

<DCOPF1> converged in 0.0106 seconds, 10 iterations with PYPOWER.


PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


True

LMP at each bus is stored in variable ``pi``

In [18]:
sp.DCOPF1.pi.v

array([0.00000776, 0.000001  , 0.00003   , 0.00001571, 0.00000917])

Kuhn-Tucker multiplier on upper and lower Pg limits are stored in ``mu1`` and ``mu2`` respectively.

In [19]:
sp.DCOPF1.mu1.v

array([0.       , 0.       , 0.       , 0.0000342, 0.       , 0.       ,
       0.       ])

In [20]:
sp.DCOPF1.mu2.v

array([0., 0., 0., 0., 0., 0., 0.])

In [21]:
sp.ACOPF1.run()

Parsing OModel for <ACOPF1>
Evaluating OModel for <ACOPF1>
Finalizing OModel for <ACOPF1>


PYPOWER Version 5.1.18, 10-Apr-2025 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011


<ACOPF1> converged in 0.1670 seconds, 15 iterations with PYPOWER.


Converged!


In ACOPF1, both active and reactive power prices are available.

In [22]:
sp.ACOPF1.pi.v

array([0.00000782, 0.000001  , 0.00003   , 0.00001561, 0.00000921])

In [23]:
sp.ACOPF1.piq.v

array([ 0.00000021, -0.        ,  0.        ,  0.0000001 ,  0.        ])

## gurobi-optimods

In this library, optimal power flow is modeled and solved using Gurobi.

<https://gurobi-optimods.readthedocs.io/en/stable/mods/opf/opf.html>

In [24]:
help(sp.OPF.run)

Help on method run in module ams.routines.grbopt:

run(**kwargs) method of ams.routines.grbopt.OPF instance
    Run the OPF routine using gurobi-optimods.

    This method invokes `self.solve(**kwargs)`, which internally utilizes
    `gurobi-optimods` to solve the OPF problem.

    Keyword arguments
    -------------------
    - ``opftype`` : str
        Type of OPF to solve (default: 'AC').
    - ``branch_switching`` : bool
        Enable branch switching (default: False).
    - ``min_active_branches`` : float
        Defines the minimum number of branches that must be turned on when
        branch switching is active, i.e. the minimum number of turned on
        branches is equal to ``numbranches * min_active_branches``. Has no
        effect if ``branch_switching`` is set to False.
    - ``use_mip_start`` : bool
        Use MIP start (default: False).
    - ``time_limit`` : float
        Time limit for the solver (default: 0.0, no limit).



In [25]:
sp.OPF.run(opftype='DC', branch_switching=True, time_limit=5)

Parsing OModel for <OPF>
Evaluating OModel for <OPF>
Finalizing OModel for <OPF>


Set parameter Username
Set parameter LicenseID to value 2617183
Set parameter TimeLimit to value 5
Set parameter OptimalityTol to value 0.0001
Academic license - for non-commercial use only - expires 2026-02-02
Building case data structures from dictionary.
Buses.
    Bus 4 ID 3 is the reference bus.
    sumloadPd 1000.0 numPload 3
    sumloadQd 328.69
    5 buses
Branches.
    Numbranches: 7 active: 7
Generators.
    Number of generators: 5
    Number of buses with gens: 5
    summaxPg 11430.0 summaxQg 10657.5
Generator cost vectors.
Running DCOPF formulation.
In bound_zs constraint, N=6
DCOPF model constructed (0.00s).
Statistics for model 'DC_Formulation_Model':
  Problem type                : MIP
  Linear constraint matrix    : 47 rows, 38 columns, 126 nonzeros
  Variable types              : 31 continuous, 7 integer (7 binary)
  Matrix range                : [1e-02, 2e+02]
  Objective range             : [1e+00, 1e+00]
  Bounds range                : [2e-01, 1e+02]
  RHS range    

<OPF> converged in 0.0436 seconds, -1 iteration with gurobi-optimods.


True

In [26]:
sp.OPF.pg.v

array([0.92351428, 0.2       , 1.        , 0.6       , 7.27648572])

As of v2.3.2, gurobi-optimods does not exposure LMP

In [27]:
sp.OPF.pi.v

array([0., 0., 0., 0., 0.])

In this routine, a decision variable ``uld`` is introduced to store the branch_switching decision.

In [28]:
sp.OPF.uld.v

array([1., 1., 1., 1., 0., 1., 1.])

In [29]:
sp.Line.u.v

array([1., 1., 1., 1., 0., 1., 1.])